![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/work-with-data/dataprep/how-to-guides/mount-dataflows.png)

# Mounting dataflows

In this notebook, you will learn how to mount a dataflow that points to a list of files in the cloud to your local compute. *Note: mounting only works on Unix or Unix-like operating systems*.

## Creating a dataflow pointing to some files
You will create a dataflow that points to some images in an Azure Blob Container. The dataflow will contain 1 column called `Path` with `StreamInfo` objects inside it, which references the source data. 

In [ ]:
import azureml.dataprep as dprep

dflow = dprep.Dataflow.get_files('https://dprepdata.blob.core.windows.net/demo/cats/*.jpg')
dflow.head(5)

Next you will create a new column that extracts the portable path of all the files.

In [ ]:
from azureml.dataprep.api.functions import get_portable_path
from azureml.dataprep.api.expressions import col

portable_path_col = 'Portable Path'
dflow = dflow.add_column(
    expression=get_portable_path(col('Path')), 
    new_column_name=portable_path_col, 
    prior_column='Path')
dflow.head(5)

You will now mount the dataflow and get the size of all the files.

In [ ]:
import sys
    
def mount_dataflow():
    import uuid
    
    from azureml.dataprep.fuse.dprepfuse import mount
    from os.path import getsize, join
    import os
    mnts_dir = '/tmp/dprepmounts/'

    mount_point = join(mnts_dir, str(uuid.uuid4()))
    os.makedirs(mount_point)

    with mount(dataflow=dflow, files_column='Path', mount_point=mount_point, foreground=False):
        mounted_files = [join(mount_point, path.lstrip('/')) for path in dflow.to_pandas_dataframe()[portable_path_col]]
        
        for file in mounted_files:
            print('{}: {}kB'.format(file, getsize(file) / 1000))
            
if sys.platform != 'linux':
    print('Mounting only works on Unix or Unix-like operating systems.')
else:
    mount_dataflow()